In [7]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#!wget https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [10]:
#!tar xf spark-3.0.1-bin-hadoop2.7.tgz

In [11]:
#!pip install -q findspark

In [12]:
#!pip install -q pyspark

In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [16]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark #verificar se há o objeto spark 

In [18]:
df = spark.read.csv("/content/drive/My Drive/dados/GO AMB/2019_det_amb.csv", header=True, sep=",")


In [63]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType


In [20]:
def to_value(v):
    try:
        return float(v.replace(",","."))
    except ValueError:
        return 0.0
udf_to_value = F.udf(to_value, FloatType())

In [21]:
dfmod= df.withColumn('VL_PROCEDIMENTO', udf_to_value(df["VL_PROCEDIMENTO"]))\
   .withColumn('QT_PROCEDIMENTO', udf_to_value(df["QT_PROCEDIMENTO"]))

In [132]:
soma = dfmod.select('CD_TUSS_PROCEDIMENTO', F.when(dfmod['VL_PROCEDIMENTO'] < 5000, 0).alias('Valor'))

In [133]:

spark.stop()